# Projeto: Lead Tech Estilista Inteligente

### Objetivo: Desenvolver um sistema de recomendação de moda utilizando um autoencoder para análise de imagens

In [ ]:
!pip install barbar
!pip install faiss-cpu

In [ ]:
# Importação de bibliotecas
import zipfile
import gc
import copy
import pickle
import time
import scipy
import cv2
import torch
import torchvision
import faiss

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import torch.nn as nn
import torch.nn.functional as F

from tqdm import tqdm
from pathlib import Path
from barbar import Bar
from ast import literal_eval
from PIL import Image, ImageDraw
from torch.utils.data import DataLoader
from torch.utils.data.dataset import Dataset
from torchvision import transforms
from torchsummary import summary

%matplotlib inline

In [ ]:
# Função para exibir uma imagem a partir do nome do arquivo
def display_image(file_name):
    try:
        img = Image.open(file_name)
        plt.imshow(img)
        plt.axis('off')
        plt.show()
    except Exception as e:
        print(f"An error occured: {e}")

# Função para visualizar os itens recomendados com base na imagem selecionada
def visualize_nearest_neighbors(selected_img_path, nearest_neighbor_paths):
    fig, axs = plt.subplots(5, 2, figsize=(10, 8))

    plt.suptitle("Recommended Items based on your selection", fontsize=16, y=1.03)

    selected_img = mpimg.imread(selected_img_path)
    axs[0, 0].imshow(selected_img)
    axs[0, 0].set_title("Item selected")
    axs[0, 0].axis('off')

    num_neighbors = min(len(nearest_neighbor_paths), 10)

    for i, ax in enumerate(axs[1:].flatten(), 1):
        if i <= num_neighbors:
            neighbor_path = nearest_neighbor_paths[i - 1]
            img = mpimg.imread(neighbor_path)
            ax.imshow(img)
            ax.set_title("Recommended Item")
            ax.axis('off')

    for i in range(5):
        axs[i, 0].axis('off')
        axs[i, 1].axis('off')

    plt.show()

In [ ]:
# Caminho dos dados de treino e validação
root_dir = 'complete-the-look-dataset/datasets/raw_train.tsv'

In [ ]:
train_df = pd.read_csv(root_dir + '/train_df.csv')
val_df = pd.read_csv(root_dir + '/val_df.csv')

In [ ]:
train_df['filename'] = root_dir + '/train_dataset/' + train_df['filename']
val_df['filename'] = root_dir + '/validation_dataset/' + val_df['filename']

In [ ]:
# Mapeamento de categorias de itens para IDs numéricos
label_name_to_id = {
    'Pants': 0,
    'Handbags': 1,
    'Shirts': 2,
    'Shoes': 3,
    'Scarves': 4,
    'Jewelry': 5,
    'Skirts': 6,
    'Coats': 7,
    'Hats': 8,
    'Dresses': 9,
    'Shorts': 10,
    'Watches': 11,
    'Sunglasses': 12,
    'Jumpsuits': 13,
    'Socks': 14,
    'Rings': 15,
    'Belts': 16,
    'Gloves': 17,
    'Swimwear': 18,
    'Stockings': 19,
    'Neckties': 20
}

train_df['label_id'] = train_df['category'].map(label_name_to_id).copy()
val_df['label_id'] = val_df['category'].map(label_name_to_id).copy()

In [ ]:
# Classe para preparar o dataset de moda com transformações necessárias
class FashionDataset(Dataset):
    def __init__(self, df):
        self.df = df
        self.transformations = transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
        ])
    def __getitem__(self, key):
        if isinstance(key, slice):
            raise NotImplementedError('slicing is not supported')
        row = self.df.iloc[key]
        try:
          image = Image.open(row['filename']).convert("RGB").resize((128, 128))
        except OSError as e:
          print(f"Error: {e}")
        image = self.transformations(image)
        return image

    def __len__(self):
        return len(self.df.index)

In [ ]:
# Criação dos conjuntos de treino e validação
train_set = FashionDataset(train_df)
val_set = FashionDataset(val_df)

In [ ]:
# Definição do modelo de autoencoder
class FeaturizerModel(nn.Module):
    def __init__(self):
        super(FeaturizerModel, self).__init__()
        self.encoder = nn.Sequential(# in- (BS,3,128, 128)

            nn.Conv2d(in_channels=3,
                      out_channels=64,
                      kernel_size=(3,3),
                      stride=1,
                      padding=1),
            nn.ReLU(True),
            nn.Conv2d(in_channels=64,
                      out_channels=64,
                      kernel_size=(3,3),
                      stride=1,
                      padding=1),
            nn.ReLU(True),
            nn.MaxPool2d(2, stride=2),

            nn.Conv2d(in_channels=64,
                      out_channels=128,
                      kernel_size=(3,3),
                      stride=2,
                      padding=1),
            nn.ReLU(True),
            nn.Conv2d(in_channels=128,
                      out_channels=128,
                      kernel_size=(3,3),
                      stride=1,
                      padding=0),
            nn.ReLU(True),
            nn.MaxPool2d(2, stride=2),

            nn.Conv2d(in_channels=128,
                      out_channels=256,
                      kernel_size=(3,3),
                      stride=2,
                      padding=1),
            nn.ReLU(True),
            nn.MaxPool2d(2, stride=2), #13

            nn.Conv2d(in_channels=256, #14
                      out_channels=512,
                      kernel_size=(3,3),
                      stride=1,
                      padding=1),
            nn.ReLU(True), # 15
            nn.Conv2d(in_channels=512, #16
                      out_channels=512,
                      kernel_size=(3,3),
                      stride=1,
                      padding=1),
            nn.Conv2d(in_channels=512,
                      out_channels=512,
                      kernel_size=(3, 3),
                      stride=1,
                      padding=0),
            nn.ReLU(True),            # 17
            nn.MaxPool2d(2, stride=2) # 18
        )
        self.decoder = nn.Sequential(

            nn.ConvTranspose2d(in_channels=512, #20
                               out_channels=512,
                               kernel_size=(3,3),
                               stride=1,
                              padding=1),
            nn.ReLU(True), #22
            nn.ConvTranspose2d(in_channels=512, #23
                               out_channels=256,
                               kernel_size=(3, 3),
                               stride=2,
                               padding=0),

            nn.ConvTranspose2d(in_channels=256, #24
                               out_channels=128,
                               kernel_size=(3,3),
                               stride=2,
                               padding=1),
            nn.ReLU(True),

            nn.ConvTranspose2d(in_channels=128,
                               out_channels=64,
                               kernel_size=(3,3),
                               stride=2,
                               padding=1),
            nn.ConvTranspose2d(in_channels=64,
                               out_channels=64,
                               kernel_size=(3,3),
                               stride=2,
                               padding=1),
            nn.ReLU(True),
            nn.ConvTranspose2d(in_channels=64,
                               out_channels=32,
                               kernel_size=(3,3),
                               stride=2,
                               padding=1),

            nn.ConvTranspose2d(in_channels=32,
                               out_channels=32,
                               kernel_size=(3,3),
                               stride=2,
                               padding=1),
            nn.ReLU(True),

            nn.ConvTranspose2d(in_channels=32,
                               out_channels=3,
                               kernel_size=(4,4),
                               stride=2,
                               padding=2),
            nn.Tanh()
        )

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x

In [ ]:
# Criação do modelo e envio para o dispositivo
featurizer_model = FeaturizerModel().to(device)

In [ ]:
# Função para carregar checkpoints do modelo
def load_checkpoint(checkpoint_fpath, model, optimizer):
    checkpoint = torch.load(checkpoint_fpath)
    model.load_state_dict(checkpoint['model_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    return model, optimizer, checkpoint['epoch']

In [ ]:
# Função para salvar checkpoints do modelo
def save_checkpoint(state, filename):
    print("=> new best achieved, chechpoint saved")
    torch.save(state, filename)

In [ ]:
# Função de treino do modelo com validação
def train_model(model, criterion, optimizer, num_epochs):
  since = time.time()
  best_model_wts = copy.deepcopy(model.state_dict())
  best_loss = np.inf

  for epoch in range(1, num_epochs + 1):
    print('Epoch {}/{}'.format(epoch, num_epochs))
    print('-' * 10)


    for phase in ['train', 'val']:
      if phase == 'train':
        model.train()
      else:
        model.eval()

      running_loss = 0.0


      for idx, inputs in enumerate(Bar(dataloaders[phase])):
        inputs = inputs.to(device)


        optimizer.zero_grad()


        with torch.set_grad_enabled(phase == 'train'):
          outputs = model(inputs)
          loss = criterion(outputs, inputs)


          if phase == 'train':
            loss.backward()
            optimizer.step()


        running_loss += loss.item() * inputs.size(0)

      epoch_loss = running_loss / dataset_sizes[phase]
      print(' {} Loss: {:.4f}'.format(phase, epoch_loss))

      if phase == 'val' and epoch_loss < best_loss:
        best_loss = epoch_loss
        best_model_wts = copy.deepcopy(model.state_dict())
        save_checkpoint(state={
            'epoch': epoch,
            'state_dict': model.state_dict(),
            'best_loss': best_loss,
            'optimizer_state_dict': optimizer.state_dict()
        }, filename='ckpt_epoch_{}.pt'.format(epoch))

    print()

  time_elapsed = time.time() - since
  print('Training complete in {:.0f}m {:.0f}s'.format(
      time_elapsed // 60, time_elapsed % 60
  ))
  print('Best val Loss: {:4f}'.format(best_loss))


  model.load_state_dict(best_model_wts)
  return model, optimizer, epoch_loss

In [ ]:
# Configurações de hiperparâmetros
EPOCHS = 40
NUM_BATCHES = 128
RETRAIN = False

dataloaders = {'train': DataLoader(train_set, batch_size=NUM_BATCHES, num_workers=2),
               'val': DataLoader(val_set, batch_size=NUM_BATCHES, num_workers=2)}

dataset_sizes = {'train': len(train_set), 'val': len(val_set)}

model = featurizer_model
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=3e-4)

In [ ]:
# Condicional para carregar o modelo
if RETRAIN == True:
    model, optimizer, state_epoch = load_checkpoint('resnet18-featurizer.pt', model, optimizer)
    print('Checkpoint Loaded')

In [ ]:
# Função para treinar o modelo
model, optimizer, loss = train_model(model=model,
                                     criterion=criterion,
                                     optimizer=optimizer,
                                     num_epochs=EPOCHS)

In [ ]:
# Salvando o estado atual do modelo
torch.save({
    'epochs': EPOCHS,
    'model_state_dict': model.state_dict(),
    'optimizer_state_dict': optimizer.state_dict(),
    'loss': loss
}, 'featurizer-model-1.pt')

In [ ]:
# Definindo as transformações de imagem para pré-processamento
transformations = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
        ])

In [ ]:
# Carrega o modelo
model = featurizer_model.to(device)
model.load_state_dict(torch.load('featurizer-model-1.pt', map_location=device)['model_state_dict'], strict=False)
model.eval()